# **Use gpt-4-turbo for sarcastic texts calssification in Ukrainian.**

### Classification is performed on test portion of data to compare with other used models

In [ ]:
%%capture
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata
import json
import pandas as pd

In [ ]:
test = pd.read_csv("test_ready_for_models.csv")

In [ ]:
test['text_mod'] = test['text_mod'].replace('\n', '').str.strip()

In [ ]:
system_setting = """Ти багато спілкуєшся онлайн. Тобі відомо, що сарказм – таке висловлювання, буквальне значення якого відрізняється від того, яке мовець насправді має на увазі.
Саркастичні тексти можуть містити такі ознаки: гіпербола, пунктуаційні знаки, прагматичні ознаки (емотикони, емоджі, великі літери), невідповідність, пародіювання російської вимови.
"""

user_input = """Класифікуй вказані тексти у зворотних лапках на один з класів: `сарказм`: 1,  `не сарказм`: 0. Поверни правильний RFC8259 JSON без жодних відхилень у такому форматі:
{{"класи": list[клас для кожного тексту]}}
1. `{0}`
2. `{1}`
3. `{2}`
4. `{3}`
5. `{4}`
6. `{5}`
7. `{6}`
8. `{7}`
9. `{8}`
10.`{9}`
"""

In [ ]:
COMPLETIONS_MODEL ="gpt-3.5-turbo"

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
import json

def parse_string(llm_output):
    """
    Parses the output of a language model and extracts labels.

    Parameters:
    llm_output (str): The output of a language model in JSON format.

    Returns:
    list: A list of labels extracted from the JSON output.
    """
    json_llm_output = json.loads(llm_output)
    labels = json_llm_output["класи"]
    return labels


In [ ]:
def request_completion_openai(system_setting, user_input):
    """
    Requests completion from the OpenAI Chat API using the provided system setting and user input.

    Parameters:
    system_setting (str): The system setting to be included in the chat completion request.
    user_input (str): The user input to be included in the chat completion request.

    Returns:
    dict: A dictionary containing the completion response from the OpenAI Chat API.
    """
    completion_response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_setting},
            {"role": "user", "content": user_input}
        ],
        model=COMPLETIONS_MODEL
    )
    return completion_response

def classify_texts_openai(messages, user_input, system_setting):
    """
    Classifies texts using the OpenAI Chat API based on provided messages, user input, and system setting.

    Parameters:
    messages (list): A list of messages to be used in formatting the user input.
    user_input (str): The user input string with formatting placeholders.
    system_setting (str): The system setting to be included in the chat completion request.

    Returns:
    str: The classification result obtained from the OpenAI Chat API.
    """
    user_input_combined = user_input.format(*messages)
    classification = request_completion_openai(system_setting, user_input_combined).choices[0].message.content.replace('\n', '')
    return classification

In [ ]:
# Perform annotation
texts = test['text_mod'].tolist()[:2150]
num_texts = len(texts)

batch_size = 10

openai_pred = []

for i in range(0, num_texts, batch_size):
     messages = texts[i:i+batch_size]
     open_ai_ouput = classify_texts_openai(messages, user_input, system_setting)
     openai_pred.append(open_ai_ouput)

In [ ]:
# Perform annotation on left texts
user_input = """Класифікуй вказані тексти у зворотних лапках на один з класів: `сарказм`: 1,  `не сарказм`: 0. Поверни правильний RFC8259 JSON без жодних відхилень у такому форматі:
{{"класи": list[клас для кожного тексту]}}
1. `{0}`
2. `{1}`
"""

texts = test['text_mod'].tolist()[2150:]
num_texts = len(texts)

batch_size = 10

openai_pred_2 = []

messages = texts
open_ai_ouput = classify_texts_openai(messages, user_input, system_setting)
openai_pred_2.append(open_ai_ouput)

In [ ]:
openai_pred.extend(openai_pred_2) #concat all predictions

In [ ]:
# Clear LLM output
cleaned_openai_pred = []

for item in openai_pred:
  try:
    cleaned_openai_pred.extend(parse_string(item))
  except:
    temp = item.replace("```json", "").replace("```", "")
    cleaned_openai_pred.extend(parse_string(temp))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(true_labels, predicted_labels):
    """
    Calculates evaluation metrics such as accuracy, precision, recall, and F1-score.

    Parameters:
    true_labels (array-like): The true labels.
    predicted_labels (array-like): The predicted labels.

    Returns:
    tuple: A tuple containing the calculated metrics in the order: (accuracy, precision, recall, f1).
    """
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    return accuracy, precision, recall, f1


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(test['is_sarcastic'].tolist(), cleaned_openai_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.5762081784386617
Precision: 0.5635658914728682
Recall: 0.6756505576208178
F1-score: 0.6145393068469991
